In [ ]:
#!/usr/bin/env python3
import os
import json
import h5py
import argparse
import pandas as pd
import numpy as np
import tinydb as db
from tinydb.storages import MemoryStorage
import matplotlib.pyplot as plt
# plt.style.use('../clint.mpl')
import matplotlib as mpl
from matplotlib.colors import LogNorm

import boost_histogram as bh
import pickle as pl

import scipy.stats as stats
import scipy.optimize as opt
from pygama import DataGroup
import pygama.lh5 as lh5
import pygama.analysis.histograms as pgh
import pygama.analysis.peak_fitting as pgf
import cage_utils
plt.rcParams['figure.figsize'] = [14, 10]

In [ ]:
run = 64 #44 #70 #64
dsp_id = '02'
user = True
hit =True
cal = True
lowE=False
etype = 'trapEftp_cal'

corr_DCR=True
corr_AoE=True
corr_ToE=True

norm = True
cut = True


dsp_list = ['energy', 'trapEftp', 'trapEmax', 'trapEftp_cal', 'bl','bl_sig', 'bl_slope', 'AoE', 'dcr', "tp_0", "tp_02", "tp_05", "tp_10", "tp_20", "tp_30", "tp_40", "tp_50", "tp_60", "tp_70", "tp_80", "tp_90", "tp_96", 'tp_max', 'ToE', 'log_tail_fit_slope', 'wf_max', 'wf_argmax', 'trapE_argmax', 'lf_max']

df_raw, dg, runtype, rt_min, radius, angle_det, rotary = cage_utils.getDataFrame(run, user=user, hit=hit, cal=cal, dsp_list=dsp_list, lowE=lowE)



In [ ]:
df_raw = df_raw.query('wf_max<15000').copy()

wflo, wfhi = 50, 15000
wf_bins = int((wfhi - wflo )/10)

median, xedges, binnumber = stats.binned_statistic(df_raw['wf_max'], df_raw['trapEftp_cal'], statistic = 'mean', bins = wf_bins) #, range = (wflo, wfhi)


en_bin_centers = pgh.get_bin_centers(xedges)

idx = np.isfinite(en_bin_centers) & np.isfinite(median)


fit_raw, cov = np.polyfit(en_bin_centers[idx], median[idx], deg=1, cov=True)

const = fit_raw[0]
offset = fit_raw[1]
err = np.sqrt(np.diag(cov))

In [ ]:
fig, ax = plt.subplots()


elo, ehi, epb = 0, 5500, 1
e_bins = int((ehi-elo)/epb)
wflo, wfhi = 0, 15000
wf_bins = int((wfhi - wflo )/10)
fig.suptitle(f'\nwf_max vs Energy', horizontalalignment='center', fontsize=16)
tp0Vtpmax_hist_norm, xedges, yedges = np.histogram2d(df_raw['wf_max'], df_raw['trapEftp_cal'], bins=[wf_bins, e_bins], range=([wflo, wfhi], [elo, ehi]))
X, Y = np.mgrid[wflo:wfhi:wf_bins*1j, elo:ehi:e_bins*1j]

pcm = plt.pcolormesh(X, Y, tp0Vtpmax_hist_norm, norm=LogNorm())
cb = plt.colorbar()
cb.set_label("counts", ha = 'right', va='center', rotation=270, fontsize=14)
cb.ax.tick_params(labelsize=12)
ax.set_xlabel('wf_max', fontsize=16)
ax.set_ylabel('trapEftp_cal (keV)', fontsize=16)
plt.setp(ax.get_xticklabels(), fontsize=12)
plt.setp(ax.get_yticklabels(), fontsize=14)
# plt.xlim(200, 1000)
# plt.ylim(0, 400)
y = const*en_bin_centers + offset
plt.plot(en_bin_centers, y, 'r')

# plt.ylim(2000, 5000)
# plt.ylim(40, 80)
# plt.xlim(50, 500)

plt.ylim(0, 100)
plt.xlim(0, 500)
plt.show()

In [ ]:
wf_max_cut = f'({const}*wf_max + {offset})*(1-0.04) < trapEftp_cal < ({const}*wf_max + {offset})*(1+0.04)'

df_raw_tot = len(df_raw)
cut_pass = len(df_raw.query(wf_max_cut).copy())
print(f'{cut_pass} events passing cut \n{(cut_pass/df_raw_tot)*100}%')

df= df_raw.query(wf_max_cut).copy()
df_60 = df.query('trapEftp_cal > 50 and trapEftp_cal < 70')

In [ ]:
fig, ax = plt.subplots()
elo, ehi, epb = 40, 80, 0.5
ToElo, ToEhi, ToEpb= 0.,  0.5, 0.0025

nbx = int((ehi-elo)/epb)
nby = int((ToEhi-ToElo)/ToEpb)



fig.suptitle(f'T/E vs Energy', horizontalalignment='center', fontsize=16)

ToE_hist_norm, xedges, yedges = np.histogram2d(df['trapEftp_cal'], df['ToE'], bins=[nbx, nby], range=([elo, ehi], [ToElo, ToEhi]))
X, Y = np.mgrid[elo:ehi:nbx*1j, ToElo:ToEhi:nby*1j]


pcm = plt.pcolormesh(X, Y, ToE_hist_norm, norm=LogNorm()) #0.002, 0.2

cb = plt.colorbar()
cb.set_label("counts", ha = 'right', va='center', rotation=270, fontsize=14)
cb.ax.tick_params(labelsize=12)
ax.set_xlabel(f'Energy (keV)', fontsize=16)
ax.set_ylabel('T/E (arb)', fontsize=16)
plt.setp(ax.get_xticklabels(), fontsize=14)
plt.setp(ax.get_yticklabels(), fontsize=14)


ax.text(0.95, 0.83, f'r = {radius} mm \ntheta = {angle_det} deg \nwf_triangle 100 ns, 10 ns', verticalalignment='bottom',
            horizontalalignment='right', transform=ax.transAxes, color='green', fontsize=14, bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})

# plt.legend()
plt.title(f'\n{runtype} run {run}, {rt_min:.2f} mins', fontsize=12)
plt.tight_layout()
# plt.ylim(-0.05, 0.1)
# plt.ylim(-0.05, 0.08)
# plt.ylim(-0.2, 0.2)
# plt.xlim(52.5, 65)
plt.show()

In [ ]:
fig, ax = plt.subplots()
blo, bhi, bpb = -10., 10., 0.05 #bl_slope
# blo, bhi, bpb = 3., 6., 0.005 #bl_sig
bl_bins = int((bhi-blo)/bpb)
ToElo, ToEhi, ToEpb= 0.,  0.5, 0.0025
ToE_bins  = int((ToEhi-ToElo)/ToEpb)
fig.suptitle(f'\nBaseline slope vs T/E', horizontalalignment='center', fontsize=16)
ToE_vs_bl, xedges, yedges = np.histogram2d(df_60['ToE'], df_60['bl_slope'], bins=[ToE_bins, bl_bins], range=([ToElo, ToEhi], [blo, bhi]))
X, Y = np.mgrid[ToElo:ToEhi:ToE_bins*1j, blo:bhi:bl_bins*1j]

pcm = plt.pcolormesh(X, Y, ToE_vs_bl, norm=LogNorm())
cb = plt.colorbar()
cb.set_label("counts", ha = 'right', va='center', rotation=270, fontsize=14)
cb.ax.tick_params(labelsize=12)
ax.set_xlabel('ToE', fontsize=16)
ax.set_ylabel('bl_slope', fontsize=16)
plt.setp(ax.get_xticklabels(), fontsize=12)
plt.setp(ax.get_yticklabels(), fontsize=14)
# plt.xlim(elo, ehi)
plt.show()

In [ ]:
fig, ax = plt.subplots()
blo, bhi, bpb = -10., 10., 0.05 #bl_slope
# blo, bhi, bpb = 3., 6., 0.005 #bl_sig
bl_bins = int((bhi-blo)/bpb)
tail_lo, tail_hi = -10e-5, -1e-5 #bl_slope
# blo, bhi, bpb = 3., 6., 0.005 #bl_sig
tail_bins = 100
fig.suptitle(f'\nTail slope vs baseline slope', horizontalalignment='center', fontsize=16)
bl_vs_tail, xedges, yedges = np.histogram2d(df_60['bl_slope'], df_60['log_tail_fit_slope'], bins=[bl_bins, tail_bins], range=([blo, bhi], [tail_lo, tail_hi]))
X, Y = np.mgrid[blo:bhi:bl_bins*1j, tail_lo:tail_hi:tail_bins*1j]

pcm = plt.pcolormesh(X, Y, bl_vs_tail, norm=LogNorm())
cb = plt.colorbar()
cb.set_label("counts", ha = 'right', va='center', rotation=270, fontsize=14)
cb.ax.tick_params(labelsize=12)
ax.set_xlabel('bl_slope', fontsize=16)
ax.set_ylabel('tail slope', fontsize=16)
plt.setp(ax.get_xticklabels(), fontsize=12)
plt.setp(ax.get_yticklabels(), fontsize=14)
# plt.xlim(elo, ehi)
plt.show()

In [ ]:
fig, ax = plt.subplots()
tail_lo, tail_hi = -10e-5, -1e-5 #bl_slope
# blo, bhi, bpb = 3., 6., 0.005 #bl_sig
tail_bins = 100
elo, ehi, epb= 50, 80, 0.25
e_bins  = int((ehi-elo)/epb)
fig.suptitle(f'\ntail slope vs energy', horizontalalignment='center', fontsize=16)
energy_vs_tail, xedges, yedges = np.histogram2d(df['trapEftp_cal'], df['log_tail_fit_slope'], bins=[e_bins, tail_bins], range=([elo, ehi], [tail_lo, tail_hi]))
X, Y = np.mgrid[elo:ehi:e_bins*1j, tail_lo:tail_hi:tail_bins*1j]

pcm = plt.pcolormesh(X, Y, energy_vs_tail, norm=LogNorm())
cb = plt.colorbar()
cb.set_label("counts", ha = 'right', va='center', rotation=270, fontsize=14)
cb.ax.tick_params(labelsize=12)
ax.set_xlabel('energy', fontsize=16)
ax.set_ylabel('tail slope', fontsize=16)
plt.setp(ax.get_xticklabels(), fontsize=12)
plt.setp(ax.get_yticklabels(), fontsize=14)
# plt.xlim(elo, ehi)
plt.show()

In [ ]:
fig, ax = plt.subplots()
blo, bhi, bpb = -10, 10., 0.5 #bl_slope
# blo, bhi, bpb = 3., 6., 0.005 #bl_sig
bl_bins = int((bhi-blo)/bpb)
elo, ehi, epb= 50, 80, 0.25
e_bins  = int((ehi-elo)/epb)
fig.suptitle(f'\nBaseline slope vs energy', horizontalalignment='center', fontsize=16)
energy_vs_bl, xedges, yedges = np.histogram2d(df['trapEftp_cal'], df['bl_slope'], bins=[e_bins, bl_bins], range=([elo, ehi], [blo, bhi]))
X, Y = np.mgrid[elo:ehi:e_bins*1j, blo:bhi:bl_bins*1j]

pcm = plt.pcolormesh(X, Y, energy_vs_bl, norm=LogNorm())
cb = plt.colorbar()
cb.set_label("counts", ha = 'right', va='center', rotation=270, fontsize=14)
cb.ax.tick_params(labelsize=12)
ax.set_xlabel('energy', fontsize=16)
ax.set_ylabel('bl_slope', fontsize=16)
plt.setp(ax.get_xticklabels(), fontsize=12)
plt.setp(ax.get_yticklabels(), fontsize=14)
# plt.xlim(elo, ehi)
plt.show()

In [ ]:
np.amax(df_60['tp_0'])

In [ ]:
fig, ax = plt.subplots()
blo, bhi, bpb = 37000, 40000, 10 #bl_slope
# blo, bhi, bpb = 3., 6., 0.005 #bl_sig
bl_bins = int((bhi-blo)/bpb)
ToElo, ToEhi, ToEpb= 0.,  0.5, 0.0025
ToE_bins  = int((ToEhi-ToElo)/ToEpb)
fig.suptitle(f'\nt0 vs T/E', horizontalalignment='center', fontsize=16)
ToE_vs_bl, xedges, yedges = np.histogram2d(df_60['ToE'], df_60['tp_0'], bins=[ToE_bins, bl_bins], range=([ToElo, ToEhi], [blo, bhi]))
X, Y = np.mgrid[ToElo:ToEhi:ToE_bins*1j, blo:bhi:bl_bins*1j]

pcm = plt.pcolormesh(X, Y, ToE_vs_bl, norm=LogNorm())
# cb = plt.colorbar()
cb.set_label("counts", ha = 'right', va='center', rotation=270, fontsize=14)
cb.ax.tick_params(labelsize=12)
ax.set_xlabel('ToE', fontsize=16)
ax.set_ylabel('t0', fontsize=16)
plt.setp(ax.get_xticklabels(), fontsize=12)
plt.setp(ax.get_yticklabels(), fontsize=14)
# plt.xlim(elo, ehi)
plt.show()

In [ ]:
fig, ax = plt.subplots()
blo, bhi, bpb = 38500, 40000, 20 #bl_slope
# blo, bhi, bpb = 3., 6., 0.005 #bl_sig
bl_bins = int((bhi-blo)/bpb)
elo, ehi, epb= 20, 100, 0.25
e_bins  = int((ehi-elo)/epb)
fig.suptitle(f'\nt0 vs energy', horizontalalignment='center', fontsize=16)
energy_vs_bl, xedges, yedges = np.histogram2d(df['trapEftp_cal'], df['tp_0'], bins=[e_bins, bl_bins], range=([elo, ehi], [blo, bhi]))
X, Y = np.mgrid[elo:ehi:e_bins*1j, blo:bhi:bl_bins*1j]

pcm = plt.pcolormesh(X, Y, energy_vs_bl, norm=LogNorm())
cb = plt.colorbar()
cb.set_label("counts", ha = 'right', va='center', rotation=270, fontsize=14)
cb.ax.tick_params(labelsize=12)
ax.set_xlabel('energy', fontsize=16)
ax.set_ylabel('t0', fontsize=16)
plt.setp(ax.get_xticklabels(), fontsize=12)
plt.setp(ax.get_yticklabels(), fontsize=14)
# plt.xlim(elo, ehi)
plt.show()

In [ ]:
fig, ax = plt.subplots()
blo, bhi, bpb = 38500, 40000, 20 #bl_slope
# blo, bhi, bpb = 3., 6., 0.005 #bl_sig
bl_bins = int((bhi-blo)/bpb)
elo, ehi, epb= 50, 5500, 10
e_bins  = int((ehi-elo)/epb)
fig.suptitle(f'\nt0 vs energy', horizontalalignment='center', fontsize=16)
energy_vs_bl, xedges, yedges = np.histogram2d(df['trapEftp_cal'], df['tp_0'], bins=[e_bins, bl_bins], range=([elo, ehi], [blo, bhi]))
X, Y = np.mgrid[elo:ehi:e_bins*1j, blo:bhi:bl_bins*1j]

pcm = plt.pcolormesh(X, Y, energy_vs_bl, norm=LogNorm())
cb = plt.colorbar()
cb.set_label("counts", ha = 'right', va='center', rotation=270, fontsize=14)
cb.ax.tick_params(labelsize=12)
ax.set_xlabel('energy', fontsize=16)
ax.set_ylabel('t0', fontsize=16)
plt.setp(ax.get_xticklabels(), fontsize=12)
plt.setp(ax.get_yticklabels(), fontsize=14)
# plt.xlim(elo, ehi)
plt.show()

In [ ]:
df['tp0max'] = df['tp_max'] - df['tp_0']

In [ ]:
fig, ax = plt.subplots()
blo, bhi, bpb = 0, 5000, 60 #bl_slope
# blo, bhi, bpb = 3., 6., 0.005 #bl_sig
bl_bins = int((bhi-blo)/bpb)
elo, ehi, epb= 50, 80, 0.5
e_bins  = int((ehi-elo)/epb)
fig.suptitle(f'\ntp0max vs energy', horizontalalignment='center', fontsize=16)
energy_vs_bl, xedges, yedges = np.histogram2d(df['trapEftp_cal'], df['tp0max'], bins=[e_bins, bl_bins], range=([elo, ehi], [blo, bhi]))
X, Y = np.mgrid[elo:ehi:e_bins*1j, blo:bhi:bl_bins*1j]

pcm = plt.pcolormesh(X, Y, energy_vs_bl, norm=LogNorm())
cb = plt.colorbar()
cb.set_label("counts", ha = 'right', va='center', rotation=270, fontsize=14)
cb.ax.tick_params(labelsize=12)
ax.set_xlabel('energy', fontsize=16)
ax.set_ylabel('tp0max', fontsize=16)
plt.setp(ax.get_xticklabels(), fontsize=12)
plt.setp(ax.get_yticklabels(), fontsize=14)
# plt.xlim(elo, ehi)
plt.show()

In [ ]:
fig, ax = plt.subplots()
blo, bhi, bpb = 0, 5000, 60 #bl_slope
# blo, bhi, bpb = 3., 6., 0.005 #bl_sig
bl_bins = int((bhi-blo)/bpb)
elo, ehi, epb= 50, 5500, 10
e_bins  = int((ehi-elo)/epb)
fig.suptitle(f'\ntp0max vs energy', horizontalalignment='center', fontsize=16)
energy_vs_bl, xedges, yedges = np.histogram2d(df['trapEftp_cal'], df['tp0max'], bins=[e_bins, bl_bins], range=([elo, ehi], [blo, bhi]))
X, Y = np.mgrid[elo:ehi:e_bins*1j, blo:bhi:bl_bins*1j]

pcm = plt.pcolormesh(X, Y, energy_vs_bl, norm=LogNorm())
cb = plt.colorbar()
cb.set_label("counts", ha = 'right', va='center', rotation=270, fontsize=14)
cb.ax.tick_params(labelsize=12)
ax.set_xlabel('energy', fontsize=16)
ax.set_ylabel('tp0max', fontsize=16)
plt.setp(ax.get_xticklabels(), fontsize=12)
plt.setp(ax.get_yticklabels(), fontsize=14)
# plt.xlim(elo, ehi)
plt.show()